### Install libraries


In [0]:
%pip install python-docx

In [0]:
%pip install Pdfplumber langchain
dbutils.library.restartPython()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

### Import Libraries

In [0]:
import os
from pyspark.sql.functions import substring_index
import pdfplumber
import docx
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [0]:


# Directory path
directory_path = "/Volumes/analytics/models/input_documents"

# List files in directory
file_paths = [file.path for file in dbutils.fs.ls(directory_path)]

# Extract file names from paths
df = spark.createDataFrame(file_paths, "string").select(substring_index("value", "/", -1).alias("file_name"))

# Show dataframe
df.show()

+--------------------+
|           file_name|
+--------------------+
|EmployeeHandbook.pdf|
|GlobalTravelPolic...|
|final-2025-tms-em...|
+--------------------+



In [0]:


pdf_volume_path =  "/Volumes/analytics/models/input_documents"  # Specify the path to the PDF volume directory

# Get the list of already processed PDF files from the Delta table
processed_files = spark.sql(f"SELECT DISTINCT file_name FROM analytics.models.docs_track").collect()
processed_files = set(row["file_name"] for row in processed_files)

# Process only new PDF files
new_files = [file for file in os.listdir(pdf_volume_path) if file not in processed_files]

all_text = ''  # Initialize all_text to store text from new PDF files

for file_name in new_files:
    file_path = os.path.join(pdf_volume_path, file_name)
    
    if file_name.lower().endswith(".pdf"):
        # Extract text from PDF
        with pdfplumber.open(file_path) as pdf:
            for pdf_page in pdf.pages:
                single_page_text = pdf_page.extract_text()
                if single_page_text:
                    all_text += '\n' + single_page_text

    elif file_name.lower().endswith(".docx"):
        # Extract text from DOCX
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            all_text += '\n' + para.text

# Split the combined text into chunks using the RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

length_function = len

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=length_function,
)
chunks = splitter.split_text(all_text)

In [0]:
df1 = spark.createDataFrame([(chunk,) for chunk in chunks], ["chunk_text"])

In [0]:
df1.createOrReplaceTempView('docs_text')

In [0]:
%sql
insert into analytics.models.docs_text (text)
select * from docs_text;

num_affected_rows,num_inserted_rows
274,274


In [0]:
df.createOrReplaceTempView("temp_table")  # Create a temporary table from the DataFrame

# Insert only the rows that do not exist in the target table
spark.sql("""
    INSERT INTO analytics.models.docs_track
    SELECT * FROM temp_table
    WHERE NOT EXISTS (
        SELECT 1 FROM analytics.models.docs_track
        WHERE temp_table.file_name = analytics.models.docs_track.file_name
    )
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]